# **SEO Product Description Generation Programme using Open AI**

In [ ]:
# If you need to use a csv file from  google drive, use the below code first.
# After the drive is successfully mounted, you can use a location from your drive.
# Otherwise upload your products.csv file to the sample_data folder in Google Colab.
# Below code should only be used if the file is not in sample_data folder.
# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


First run the below code from the terminal:

pip install pandas openai

Then copy your products.csv file to the sample_data folder in Google Colab.


⭐ **With the below code, unfinished sentence issue has been fixed.** ⭐

In [2]:
import pandas as pd
from openai import OpenAI
import os
from google.colab import userdata

# Get User Secret Key defined in Google Colab
api_key = userdata.get('OpenAI_Colab_SEO')

# Initialize the OpenAI client with your API key
client = OpenAI(api_key=api_key)

# Function to generate description using OpenAI's API
def generate_description(name, category, keywords):
    prompt = (
        f"Generate a product description for a product in British English with the following details:\n"
        f"Name: {name}\n"
        f"Category: {category}\n"
        f"Keywords: {', '.join(keywords)}\n"
    )

    # Call the OpenAI API to generate the completion
    completion = client.chat.completions.create(
        model="gpt-4",  # specify the GPT model
        max_tokens=200,  # set the maximum number of tokens to control length (adjust as needed)
        messages=[
            {"role": "system", "content": "You are an SEO content specialist."},
            {"role": "user", "content": prompt}
        ]
    )

    # Retrieve the generated text from the completion
    generated_text = completion.choices[0].message.content.strip()

    # Check if the last sentence ends with a period
    if generated_text.endswith('.') and '.' in generated_text[:-1]:
        return generated_text
    else:
        # Remove the last incomplete sentence
        last_period_index = generated_text.rfind('.')
        if last_period_index != -1:
            generated_text = generated_text[:last_period_index + 1]
        return generated_text

# Function to process products and update CSV file
def process_products(csv_file):
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)

    # Generate descriptions for each product
    for index, row in df.iterrows():
        sku = row['SKU']
        name = row['Product Name']
        category = row['Product Category']
        keywords = row['Keywords']

        # Split keywords into a list
        keywords_list = [kw.strip() for kw in keywords.split(',')]

        # Generate product description using OpenAI API
        description = generate_description(name, category, keywords_list)

        # Update the DataFrame with the generated description
        df.at[index, 'Description'] = description

    # Save the updated DataFrame to a CSV file
    updated_csv_file = '/content/sample_data/updated_products.csv'
    df.to_csv(updated_csv_file, index=False)

# Example usage
if __name__ == "__main__":
    # Path to your CSV file in Google Drive
    csv_file = '/content/sample_data/products.csv'

    # Process the products and update the CSV file
    process_products(csv_file)
